In [1]:
%matplotlib inline

import pandas as pd
import nivapy3 as nivapy
import geopandas as gpd
import matplotlib.pyplot as plt
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import text

plt.style.use('ggplot')

# Add catchment data to DSToolkit PostGIS database

In [2]:
## Connect to PostGIS
## If connecting from JupyterHub
# eng = nivapy.da.connect_postgis(admin=True)

In [3]:
# Connect to PostGIS
# If connecting from "local" Docker
eng = nivapy.da.connect_postgis(
    admin=True, host="104.199.55.41", database="general", port=5432
)

Username:  ········
Password:  ·······


Connection successful.


## 1. Create database structure

### 1.1. Create Schema

In [4]:
sql = "CREATE SCHEMA IF NOT EXISTS niva"
eng.execute(sql)

In [6]:
# Grant "ReadOnly" privileges to default Jovyan user
sql_list = [ 
    'GRANT USAGE ON SCHEMA niva TO jovyan',   
    'GRANT SELECT ON ALL TABLES IN SCHEMA niva TO jovyan',    
    'ALTER DEFAULT PRIVILEGES IN SCHEMA niva GRANT SELECT ON TABLES TO jovyan',
]

for sql in sql_list:
    eng.execute(sql)

### 1.2. Create tables

#### 1.2.1. Stations

In [7]:
# Delete if already exists
sql = ("DROP TABLE IF EXISTS niva.stations")
eng.execute(sql)

# Create table
sql = ("CREATE TABLE niva.stations "
       "( "
       "  station_id SERIAL PRIMARY KEY, "
       "  station_code text UNIQUE NOT NULL, "
       "  station_name text UNIQUE NOT NULL, "
       "  aquamonitor_id integer UNIQUE, "
       "  longitude numeric NOT NULL, "
       "  latitude numeric NOT NULL, "
       "  geom geometry(Point, 4326) NOT NULL "
       ")")
eng.execute(sql)

sql = ("CREATE INDEX niva_stations_spidx "
       "ON niva.stations "
       "USING GIST (geom)")
eng.execute(sql)

#### 1.2.2. Projects

In [8]:
# Delete if already exists
sql = ("DROP TABLE IF EXISTS niva.projects")
eng.execute(sql)

# Create table
sql = ("CREATE TABLE niva.projects "
       "( "
       "  project_id SERIAL PRIMARY KEY, "
       "  project_code text UNIQUE NOT NULL, "
       "  project_name text UNIQUE NOT NULL, "
       "  aquamonitor_id integer UNIQUE, "
       "  contact text, "
       "  description text "
       ")")
eng.execute(sql)

#### 1.2.3. Projects-stations

In [9]:
# Delete if already exists
sql = ("DROP TABLE IF EXISTS niva.projects_stations")
eng.execute(sql)

# Create table
sql = ("CREATE TABLE niva.projects_stations "
       "( "
       "  project_id integer NOT NULL, "
       "  station_id integer NOT NULL, "
       "  PRIMARY KEY (project_id, station_id), "
       "  CONSTRAINT project_id FOREIGN KEY (project_id) "
       "      REFERENCES niva.projects (project_id) "
       "      ON UPDATE NO ACTION ON DELETE NO ACTION, "
       "  CONSTRAINT station_id_fkey FOREIGN KEY (station_id) "
       "      REFERENCES niva.stations (station_id) "
       "      ON UPDATE NO ACTION ON DELETE NO ACTION "
       ")")
eng.execute(sql)

#### 1.2.4. Catchments

In [10]:
# Delete if already exists
sql = ("DROP TABLE IF EXISTS niva.catchments")
eng.execute(sql)

# Create table
sql = ("CREATE TABLE niva.catchments "
       "( "
       "  station_id integer NOT NULL, "
       "  geom geometry(Multipolygon, 4326) NOT NULL, "
       "  PRIMARY KEY (station_id), "
       "  CONSTRAINT station_id_fkey FOREIGN KEY (station_id) "
       "      REFERENCES niva.stations (station_id) "
       "      ON UPDATE NO ACTION ON DELETE NO ACTION "
       ")")
eng.execute(sql)

sql = ("CREATE INDEX niva_catchments_spidx "
       "ON niva.catchments "
       "USING GIST (geom)")
eng.execute(sql)

## 2. Add stations and projects

In [11]:
# Read projects
xl_path = r"./stn_proj_catch_templates/refelv_over_stns.xlsx"
proj_df = pd.read_excel(xl_path, sheet_name="project")
assert len(proj_df) == 1

proj_df

,project_code,project_name,aquamonitor_id,contact,description
0,Refelv_Over,Overvåking av referanseelver,11226,NaN,Overvåking av referanseelver


In [12]:
# Add project
proj_df.to_sql('projects',
               schema='niva',
               con=eng,
               if_exists='append',
               index=False,
              )

### 2.2. Add stations

In [13]:
# Read stations
xl_path = r"./stn_proj_catch_templates/refelv_over_stns.xlsx"
stn_df = pd.read_excel(xl_path, sheet_name="stations")
stn_df["longitude"] = stn_df["longitude"].round(6)
stn_df["latitude"] = stn_df["latitude"].round(6)

stn_df.head()

,station_code,station_name,aquamonitor_id,longitude,latitude,fpath_or_id
0,S_019-242_Asl,Aslestadåi,69564,7.928927,59.310474,68
1,O_002-305_Atn_DAN04,"Atna nedstrøms Atnasjøen, DAN04",66479,10.233701,61.850606,33
2,O_002-300_Atn_DAN03,"Atna v/ Elgvassli, DAN03",66478,10.012606,62.007041,32
3,O_002-305_Atn_DAN11,"Atna, DAN11",66483,10.746483,61.745664,34
4,V_073-78_Sme,Bekkefelt nedre del av Smedalselvi og Mørkedøla,69553,7.965413,61.072394,11


In [14]:
# Build geom
stn_gdf = gpd.GeoDataFrame(
    stn_df,
    crs={"init": "epsg:4326"},
    geometry=gpd.points_from_xy(stn_df.longitude, stn_df.latitude),
).copy()
stn_gdf["geom"] = stn_gdf["geometry"].apply(lambda x: WKTElement(x.wkt, srid=4326))
del stn_gdf["fpath_or_id"], stn_gdf["geometry"]

stn_gdf.head()

,station_code,station_name,aquamonitor_id,longitude,latitude,geom
0,S_019-242_Asl,Aslestadåi,69564,7.928927,59.310474,POINT (7.928927 59.310474)
1,O_002-305_Atn_DAN04,"Atna nedstrøms Atnasjøen, DAN04",66479,10.233701,61.850606,POINT (10.233701 61.850606)
2,O_002-300_Atn_DAN03,"Atna v/ Elgvassli, DAN03",66478,10.012606,62.007041,POINT (10.012606 62.007041)
3,O_002-305_Atn_DAN11,"Atna, DAN11",66483,10.746483,61.745664,POINT (10.746483 61.745664)
4,V_073-78_Sme,Bekkefelt nedre del av Smedalselvi og Mørkedøla,69553,7.965413,61.072394,POINT (7.965413 61.072394)


In [15]:
# Add stations
stn_gdf.to_sql('stations',
               schema='niva',
               con=eng,
               if_exists='append',
               index=False,
               dtype={'geom': Geometry('POINT', srid=4326)},
               method='multi',
               chunksize=1000,
              )

### 2.3. Add project-stations

In [16]:
# Build table for projects-stations
# Get station IDs
stn_codes = tuple(stn_df["station_code"].unique())
sql = text("SELECT station_id FROM niva.stations " 
           "WHERE station_code IN :stn_codes")
prst_df = pd.read_sql(sql, params={"stn_codes": stn_codes}, con=eng)

# Get project ID
proj_code = proj_df["project_code"].values[0]
sql = text("SELECT project_id FROM niva.projects " 
           "WHERE project_code = :proj_code")
proj_id = pd.read_sql(sql, params={"proj_code": proj_code}, con=eng)["project_id"].iloc[0]

prst_df["project_id"] = proj_id
prst_df.head()

,station_id,project_id
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


In [17]:
# Add projects-stations
prst_df.to_sql('projects_stations',
               schema='niva',
               con=eng,
               if_exists='append',
               index=False,
               method='multi',
               chunksize=1000,
              )

### 2.4. Add catchments

In [2]:
# Connect to Jose's PostGIS database
# Must be from "local" Docker
jlg_eng = nivapy.da.connect_postgis(
    host="34.89.139.67",                       # IP is not statisc - get latest from José
    database="geonorway",
    port=5432,
    user="james",
    password="jamesrocks",  # Thanks Jose ;-)
)

Connection failed.
(psycopg2.OperationalError) could not connect to server: Connection refused
	Is the server running on host "34.89.139.67" and accepting
	TCP/IP connections on port 5432?

(Background on this error at: http://sqlalche.me/e/e3q8)


In [19]:
# Read catchments
sql = (
    "SELECT station_id, "
    "  ST_Transform(ST_Multi(basin), 4326) AS geom "
    "FROM metno.resultsshp"
)
cat_gdf = gpd.read_postgis(sql, jlg_eng, geom_col="geom")
cat_gdf.rename({"station_id": "fpath_or_id"}, axis=1, inplace=True)

# Save
# cat_gdf.to_file('jose_catchments.geojson', driver='GeoJSON')

cat_gdf.head()

,fpath_or_id,geom
0,15,"MULTIPOLYGON (((8.01595 61.55096, 8.01642 61.5..."
1,52,"MULTIPOLYGON (((12.24675 64.71368, 12.24779 64..."
2,17,"MULTIPOLYGON (((7.64513 60.53337, 7.64559 60.5..."
3,24,"MULTIPOLYGON (((11.28994 62.05309, 11.29327 62..."
4,26,"MULTIPOLYGON (((10.01309 61.84945, 10.02823 61..."


In [20]:
# Get station IDs from db
sql = text(
    "SELECT station_id, station_code FROM niva.stations "
    "WHERE station_id IN ( "
    "  SELECT station_id from niva.projects_stations "
    "  WHERE project_id = :proj_id)"
)
stn_ids = pd.read_sql(sql, params={"proj_id": 1}, con=eng)

# Join José's IDs
df = pd.merge(
    stn_ids, stn_df[["station_code", "fpath_or_id"]], how="left", on="station_code",
)

# Join catchments
cat_gdf = cat_gdf.merge(df, on="fpath_or_id")

# Tidy
cat_gdf = cat_gdf[["station_id", "geom"]]
cat_gdf["geom"] = cat_gdf["geom"].apply(lambda x: WKTElement(x.wkt, srid=4326))

cat_gdf.head()

,station_id,geom
0,73,MULTIPOLYGON (((8.01594911052848 61.5509631906...
1,46,MULTIPOLYGON (((12.24675357327684 64.713675950...
2,47,MULTIPOLYGON (((7.645134072621154 60.533369423...
3,43,MULTIPOLYGON (((11.28993999286648 62.053088792...
4,59,MULTIPOLYGON (((10.01309118466988 61.849447372...


In [21]:
# Add catchments
cat_gdf.to_sql('catchments',
               schema='niva',
               con=eng,
               if_exists='append',
               index=False,
               dtype={'geom': Geometry('MULTIPOLYGON', srid=4326)},
               method='multi',
               chunksize=1000,
              )